**IMPORT LIBRARIES AND CONSTANTS**

In [ ]:
import os
import shutil
import subprocess
import cv2
import git
import tqdm
import time
import random

PROJECT_FOLDER_ABS_PATH = os.path.dirname(os.path.abspath("__file__")).replace(os.sep, "/")

from CNN_OD_constants import Constants

const = Constants(PROJECT_FOLDER_ABS_PATH)

import CNN_OD_xml_to_txt

**PREPARE SELECTED SHOT IMAGES**

In [ ]:
if not os.path.exists(const.SELECTED_PREPARED_IMAGES_FOLDER_ABS_PATH):
    os.makedirs(const.SELECTED_PREPARED_IMAGES_FOLDER_ABS_PATH)

const.SELECTED_CLASS_NAMES_LIST = os.listdir(const.SELECTED_SHOT_IMAGES_FOLDER_ABS_PATH)

for class_name in tqdm.tqdm(const.SELECTED_CLASS_NAMES_LIST):
    const.SELECTED_CLASSIFIED_SHOT_IMAGES_FOLDER_ABS_PATH = const.SELECTED_SHOT_IMAGES_FOLDER_ABS_PATH + "/" + class_name
    const.SELECTED_CLASSIFIED_PREPARED_IMAGES_FOLDER_ABS_PATH = const.SELECTED_PREPARED_IMAGES_FOLDER_ABS_PATH + "/" + class_name
    
    if not os.path.exists(const.SELECTED_CLASSIFIED_PREPARED_IMAGES_FOLDER_ABS_PATH):
        os.makedirs(const.SELECTED_CLASSIFIED_PREPARED_IMAGES_FOLDER_ABS_PATH)
       
    for shot_image_name in os.listdir(const.SELECTED_CLASSIFIED_SHOT_IMAGES_FOLDER_ABS_PATH):
        if shot_image_name.endswith(".jpg"):
            shot_image = cv2.imread(const.SELECTED_CLASSIFIED_SHOT_IMAGES_FOLDER_ABS_PATH + "/" + shot_image_name)

            if shot_image.shape[0] > shot_image.shape[1]:
                oriented_shot_image = cv2.rotate(shot_image, cv2.ROTATE_90_CLOCKWISE)
            else:
                oriented_shot_image = shot_image.copy()
            prepared_image = cv2.resize(oriented_shot_image, const.PREPARED_IMAGE_SIZE, interpolation=cv2.INTER_AREA)
            cv2.imwrite(const.SELECTED_CLASSIFIED_PREPARED_IMAGES_FOLDER_ABS_PATH + "/" + shot_image_name, prepared_image)

**SETUP LABELIMG AND LABEL SELECTED PREPARED IMAGES**

In [ ]:
if not os.path.exists(const.LABELIMG_FOLDER_ABS_PATH):
    git.Repo.clone_from(const.LABELIMG_URL, const.LABELIMG_FOLDER_ABS_PATH)
    activating_process = subprocess.Popen("cmd.exe /c cd " + chr(34) + const.LABELIMG_FOLDER_ABS_PATH + chr(34) + " && pyrcc5 -o libs/resources.py resources.qrc")
    time.sleep(1)

const.SELECTED_CLASS_NAMES_LIST = os.listdir(const.SELECTED_PREPARED_IMAGES_FOLDER_ABS_PATH)

for class_name in const.SELECTED_CLASS_NAMES_LIST:
    const.SELECTED_CLASSIFIED_XML_LABELS_FOR_PREPARED_IMAGES_FOLDER_ABS_PATH = const.SELECTED_XML_LABELS_FOR_PREPARED_IMAGES_FOLDER_ABS_PATH + "/" + class_name
    
    if not os.path.exists(const.SELECTED_CLASSIFIED_XML_LABELS_FOR_PREPARED_IMAGES_FOLDER_ABS_PATH):
        os.makedirs(const.SELECTED_CLASSIFIED_XML_LABELS_FOR_PREPARED_IMAGES_FOLDER_ABS_PATH)

starting_process = subprocess.Popen("cmd.exe /c cd " + chr(34) + const.LABELIMG_FOLDER_ABS_PATH + chr(34) + " && python labelImg.py")

**CONVERT XML LABELS TO TXT LABELS USED WITH YOLO**

In [ ]:
label_names_and_ids_dict = dict()

for (genuine_class_id, genuine_class_name) in enumerate(const.SELECTED_GENUINE_CLASS_NAMES_LIST, start=0):
    label_names_and_ids_dict[genuine_class_name] = genuine_class_id

const.SELECTED_CLASS_NAMES_LIST = os.listdir(const.SELECTED_PREPARED_IMAGES_FOLDER_ABS_PATH)

for class_name in tqdm.tqdm(const.SELECTED_CLASS_NAMES_LIST):
    const.SELECTED_CLASSIFIED_PREPARED_IMAGES_FOLDER_ABS_PATH = const.SELECTED_PREPARED_IMAGES_FOLDER_ABS_PATH + "/" + class_name
    const.SELECTED_CLASSIFIED_XML_LABELS_FOR_PREPARED_IMAGES_FOLDER_ABS_PATH = const.SELECTED_XML_LABELS_FOR_PREPARED_IMAGES_FOLDER_ABS_PATH + "/" + class_name
    const.SELECTED_CLASSIFIED_TXT_LABELS_FOR_PREPARED_IMAGES_FOLDER_ABS_PATH = const.SELECTED_TXT_LABELS_FOR_PREPARED_IMAGES_FOLDER_ABS_PATH + "/" + class_name

    const.SELECTED_CLASSIFIED_XML_LABELS_FOR_PREPARED_IMAGES_LIST = os.listdir(const.SELECTED_CLASSIFIED_XML_LABELS_FOR_PREPARED_IMAGES_FOLDER_ABS_PATH)

    if not os.path.exists(const.SELECTED_CLASSIFIED_TXT_LABELS_FOR_PREPARED_IMAGES_FOLDER_ABS_PATH):
        os.makedirs(const.SELECTED_CLASSIFIED_TXT_LABELS_FOR_PREPARED_IMAGES_FOLDER_ABS_PATH)
    
    for xml_label_name in const.SELECTED_CLASSIFIED_XML_LABELS_FOR_PREPARED_IMAGES_LIST:
        CNN_OD_xml_to_txt.convert_xml_to_txt(xml_label_name, label_names_and_ids_dict, const.SELECTED_CLASSIFIED_XML_LABELS_FOR_PREPARED_IMAGES_FOLDER_ABS_PATH, \
            const.SELECTED_CLASSIFIED_TXT_LABELS_FOR_PREPARED_IMAGES_FOLDER_ABS_PATH)

**PREPARE SELECTED LABELED SECTIONS**

In [ ]:
for selected_section_folder_abs_path in [const.SELECTED_TRAINING_SECTION_IMAGES_FOLDER_ABS_PATH, const.SELECTED_TRAINING_SECTION_LABELS_FOLDER_ABS_PATH, \
    const.SELECTED_TESTING_SECTION_IMAGES_FOLDER_ABS_PATH, const.SELECTED_TESTING_SECTION_LABELS_FOLDER_ABS_PATH, const.SELECTED_VALIDATION_SECTION_IMAGES_FOLDER_ABS_PATH, \
        const.SELECTED_VALIDATION_SECTION_LABELS_FOLDER_ABS_PATH]:
    if os.path.exists(selected_section_folder_abs_path):
        shutil.rmtree(selected_section_folder_abs_path)
    os.makedirs(selected_section_folder_abs_path)

const.SELECTED_CLASS_NAMES_LIST = os.listdir(const.SELECTED_PREPARED_IMAGES_FOLDER_ABS_PATH)

for class_name in tqdm.tqdm(const.SELECTED_CLASS_NAMES_LIST):
    const.SELECTED_CLASSIFIED_PREPARED_IMAGES_FOLDER_ABS_PATH = const.SELECTED_PREPARED_IMAGES_FOLDER_ABS_PATH + "/" + class_name
    const.SELECTED_CLASSIFIED_TXT_LABELS_FOR_PREPARED_IMAGES_FOLDER_ABS_PATH = const.SELECTED_TXT_LABELS_FOR_PREPARED_IMAGES_FOLDER_ABS_PATH + "/" + class_name
    const.SELECTED_CLASSIFIED_PREPARED_IMAGE_NAMES_LIST = os.listdir(const.SELECTED_CLASSIFIED_PREPARED_IMAGES_FOLDER_ABS_PATH)

    for prepared_image_name in const.SELECTED_CLASSIFIED_PREPARED_IMAGE_NAMES_LIST:
        if prepared_image_name.endswith(".jpg"):
            txt_label_for_prepared_image = prepared_image_name.replace(".jpg", ".txt")
            shutil.copy(const.SELECTED_CLASSIFIED_PREPARED_IMAGES_FOLDER_ABS_PATH + "/" + prepared_image_name, const.SELECTED_TRAINING_SECTION_IMAGES_FOLDER_ABS_PATH)
            shutil.copy(const.SELECTED_CLASSIFIED_TXT_LABELS_FOR_PREPARED_IMAGES_FOLDER_ABS_PATH + "/" + txt_label_for_prepared_image, \
                const.SELECTED_TRAINING_SECTION_LABELS_FOLDER_ABS_PATH)
    
    const.SELECTED_TESTING_AND_VALIDATION_SECTION_IMAGES_NAMES_LIST = random.sample(const.SELECTED_CLASSIFIED_PREPARED_IMAGE_NAMES_LIST, \
        int(round((const.TESTING_PERCENTAGE_OF_DATASET + const.VALIDATION_PERCENTAGE_OF_DATASET) * len(const.SELECTED_CLASSIFIED_PREPARED_IMAGE_NAMES_LIST))))

    for random_prepared_image_name in const.SELECTED_TESTING_AND_VALIDATION_SECTION_IMAGES_NAMES_LIST:
        if random_prepared_image_name.endswith(".jpg"):
            txt_label_for_random_prepared_image = random_prepared_image_name.replace(".jpg", ".txt")
            shutil.move(const.SELECTED_TRAINING_SECTION_IMAGES_FOLDER_ABS_PATH + "/" + random_prepared_image_name, const.SELECTED_TESTING_SECTION_IMAGES_FOLDER_ABS_PATH + "/" + \
                random_prepared_image_name)
            shutil.move(const.SELECTED_TRAINING_SECTION_LABELS_FOLDER_ABS_PATH + "/" + txt_label_for_random_prepared_image, \
                const.SELECTED_TESTING_SECTION_LABELS_FOLDER_ABS_PATH + "/" + txt_label_for_random_prepared_image)
    
    const.SELECTED_VALIDATION_SECTION_IMAGES_NAMES_LIST = random.sample(const.SELECTED_TESTING_AND_VALIDATION_SECTION_IMAGES_NAMES_LIST, \
        int(round(const.VALIDATION_PERCENTAGE_OF_DATASET / (const.TESTING_PERCENTAGE_OF_DATASET + const.VALIDATION_PERCENTAGE_OF_DATASET) * \
            len(const.SELECTED_TESTING_AND_VALIDATION_SECTION_IMAGES_NAMES_LIST))))
    
    for random_prepared_image_name in const.SELECTED_VALIDATION_SECTION_IMAGES_NAMES_LIST:
        if random_prepared_image_name.endswith(".jpg"):
            txt_label_for_random_prepared_image = random_prepared_image_name.replace(".jpg", ".txt")
            shutil.move(const.SELECTED_TESTING_SECTION_IMAGES_FOLDER_ABS_PATH + "/" + random_prepared_image_name, const.SELECTED_VALIDATION_SECTION_IMAGES_FOLDER_ABS_PATH + \
                "/" + random_prepared_image_name)
            shutil.move(const.SELECTED_TESTING_SECTION_LABELS_FOLDER_ABS_PATH + "/" + txt_label_for_random_prepared_image, \
                const.SELECTED_VALIDATION_SECTION_LABELS_FOLDER_ABS_PATH + "/" + txt_label_for_random_prepared_image)

**ZIP SELECTED LABELED SECTIONS AND COPY OD CONSTANTS FOR SAVING ON GOOGLE DRIVE**

In [ ]:
ZIPPED_SELECTED_LABELED_SECTIONS_FILE_NAME = (const.LABELED_SECTIONS_FOLDER_REL_PATH + " - " + const.SELECTED_DATASET_NAME + " dataset").replace(" ", "_")
ZIPPED_SELECTED_LABELED_SECTIONS_FILE_ABS_PATH = const.GOOGLE_DRIVE_FILES_FOLDER_ABS_PATH + "/" + ZIPPED_SELECTED_LABELED_SECTIONS_FILE_NAME

if not os.path.exists(const.GOOGLE_DRIVE_FILES_FOLDER_ABS_PATH):
    os.makedirs(const.GOOGLE_DRIVE_FILES_FOLDER_ABS_PATH)

shutil.make_archive(ZIPPED_SELECTED_LABELED_SECTIONS_FILE_ABS_PATH, "zip", const.SELECTED_LABELED_SECTIONS_FOLDER_ABS_PATH)

shutil.copy(const.OD_CONSTANTS_FILE_ABS_PATH, const.GOOGLE_DRIVE_FILES_FOLDER_ABS_PATH)

**SETUP YOLO MODELS**

In [ ]:
if not os.path.exists(const.YOLO_MODELS_FOLDER_ABS_PATH):
  git.Repo.clone_from(const.YOLO_MODELS_URL, const.YOLO_MODELS_FOLDER_ABS_PATH)

activating_process = subprocess.Popen("cmd.exe /c cd " + chr(34) + const.YOLO_MODELS_FOLDER_ABS_PATH + chr(34) + " && pip install -qr requirements.txt")

**UNZIP ZIPPED TRAINED YOLO MODEL FOLDER**

In [ ]:
# trained YOLO model folder name changes for every new trained model

# CHANGEABLE VARIABLES:

SELECTED_TRAINED_YOLO_MODEL_FOLDER_NAME = "household_items_model_640DIM_0,005LR_100EP"

# # #


SELECTED_ZIPPED_TRAINED_YOLO_MODEL_FILE_NAME = SELECTED_TRAINED_YOLO_MODEL_FOLDER_NAME
SELECTED_ZIPPED_TRAINED_YOLO_MODEL_FILE_ABS_PATH = PROJECT_FOLDER_ABS_PATH + "/" + SELECTED_ZIPPED_TRAINED_YOLO_MODEL_FILE_NAME

SELECTED_MODEL_TRAINED_ON_COLAB_FOLDER_REL_PATH = const.MODELS_TRAINED_ON_COLAB_FOLDER_REL_PATH + "/" + SELECTED_TRAINED_YOLO_MODEL_FOLDER_NAME
SELECTED_MODEL_TRAINED_ON_COLAB_FOLDER_ABS_PATH = PROJECT_FOLDER_ABS_PATH + "/" + SELECTED_MODEL_TRAINED_ON_COLAB_FOLDER_REL_PATH

SELECTED_TRAINED_YOLO_MODEL_FOLDER_REL_PATH = "runs/train/" + SELECTED_TRAINED_YOLO_MODEL_FOLDER_NAME
SELECETD_TRAINED_YOLO_MODEL_FOLDER_ABS_PATH = const.YOLO_MODELS_FOLDER_ABS_PATH + "/" + SELECTED_TRAINED_YOLO_MODEL_FOLDER_REL_PATH

if os.path.exists(SELECTED_ZIPPED_TRAINED_YOLO_MODEL_FILE_ABS_PATH + ".zip"):
    if not os.path.exists(SELECTED_MODEL_TRAINED_ON_COLAB_FOLDER_ABS_PATH):
        os.makedirs(SELECTED_MODEL_TRAINED_ON_COLAB_FOLDER_ABS_PATH)
    
    if not os.path.exists(SELECETD_TRAINED_YOLO_MODEL_FOLDER_ABS_PATH):
        os.makedirs(SELECETD_TRAINED_YOLO_MODEL_FOLDER_ABS_PATH)
    
    if not os.path.exists(const.MODELS_TRAINED_ON_COLAB_FOLDER_ABS_PATH):
        os.makedirs(const.MODELS_TRAINED_ON_COLAB_FOLDER_ABS_PATH)
    
    shutil.unpack_archive(SELECTED_ZIPPED_TRAINED_YOLO_MODEL_FILE_ABS_PATH + ".zip", SELECTED_MODEL_TRAINED_ON_COLAB_FOLDER_ABS_PATH, "zip")
    shutil.unpack_archive(SELECTED_ZIPPED_TRAINED_YOLO_MODEL_FILE_ABS_PATH + ".zip", SELECETD_TRAINED_YOLO_MODEL_FOLDER_ABS_PATH, "zip")
    # os.remove(SELECTED_ZIPPED_TRAINED_YOLO_MODEL_FILE_ABS_PATH + ".zip")